# MTH5 Example 07

## Make an MTH5 from Phoenix Data

This example demonstrates how to read Phoenix data into an MTH5 file.  These data are provided by Phoenix Geophysical Ltd. Provided are 3 synchronously recored stations.  The Phoenix instruments MTU-5 series are broadband instruments with multiple sample rates.  

The instrumnet is constantly recording at 24 kHz and depending on the input paramters those data are decimated or chunked into different data files.  A common setup is to record continuously at 150 samples per second and record short bursts of 24 kHz for just a few seconds every 15 minutes or so.  In this case the continuous 24 kHz is decimated down to 150 Hz and stored in files with a length of 6 minutes.  The short bursts are also stored into a separate file.  This is done for each channel recorded.  Its common for broadband instruments to record each channel individually making it easier to buffer.  

A challenge arises when you have the short bursts is that you will have a ton of runs.  Luckily HDF5 can handle this, we just have to organize it properly.  And figure out which file belongs where.  A `PhoenixCollection` object is provided for just this task. 

## Imports

In [1]:
from pathlib import Path

from mth5.clients import MakeMTH5

## Data Directory

Specify the station directory.  Phoenix files place each channel in a folder under the station directory named by the channel number.  There is also a `recmeta.json` file that has metadata output by the receiver that can be useful.  In the `PhoenixGeopPy/sample_data` there are 2 folders one for native data, these are `.bin` files which are the raw data in counts sampled at 24k.  There is also a folder for segmented files, these files are calibrated to millivolts and decimated or segmented data according to the recording configuration.  Most of the time you would use the segmented files? 

In [3]:
source_dir = Path.home().joinpath("shared", "shortcourses", "mt", "phoenix")
target_dir = Path().cwd().parent.parent.joinpath("data", "timeseries", "phoenix")
target_dir.mkdir(parents=True, exist_ok=True)
print(f"Source directory: {source_dir}")
print(f"Target directory: {target_dir.absolute()}")


Source directory: /home/kkappler/shared/shortcourses/mt/phoenix
Target directory: /home/kkappler/software/irismt/earthscope-mt-course/data/timeseries/phoenix


## Make an MTH5 using MakeMTH5 Clients

A convenience class has been developed to make it easier to build MTH5 from data.  Here we will demonstrate how one would build an MTH5 from Phoenix MTU-5c data.  The important things you need to do if you are collecting data with Phoenix instruments or working with Phoenix data is to export the receiver and sensor calibration files using the Phoenix software EMPower.  Export these into a folder that makes logical sens to you.  A common folder name would be `calibrations`.

The classmethod `MakeMTH5.from_phoenix` will look in the data folder for the data files.  Phoenix organizes the data into folder for each channel and data are cached every 6 minutes usually, this will organize those files into the longest possible continuous runs for the 150 samples/second data.  Similarly for the 24k samples/second data runs will be created for each burst, so you will have a lot of them.  

You will give it the `sample_rates` that you would like to archive in the MTH5.

`receiver_calibration_dict` can be a dictionary where keys are the receiver ID numbers and the values are the paths to the `rxcal.json` files, **or** the simplest way is to give it a path to the location of the `rxcal.json` files you exported from EMPower and the code will sort them out and match them with the appropriate channel using the `recmeta.json` file.

`sensor_calibration_dict` can be a dictionary where keys are the sensor ID numbers and the values are `PhoenixCalibration` objects, **or** the simplest is to give it a path to the `scal.json` files exported from EMPower.  The code will match the calibrations to the appropriate channel using the information in `recmeta.json` file.

You can set the `mth5_filename` to something useful, default is `from_phoenix.h5`.

You can set the `save_path` which can be the full path to the new H5 file or the directory to save to.

## Local Station

Here we create an MTH5 file for the local station.

In [6]:
local_station = "9043"
local_station_dir = source_dir.joinpath(local_station)

phx_mth5_fn = MakeMTH5.from_phoenix(
    local_station_dir,
    save_path=target_dir,
    mth5_filename=f"from_phoenix_{local_station}.h5",
    sample_rates=[150, 24000],
    receiver_calibration_dict=Path(local_station_dir.joinpath("calibrations")),
    sensor_calibration_dict=Path(local_station_dir.joinpath("calibrations")),
)

2024-10-17T11:41:38.128365-0700 | INFO | mth5.mth5 | _initialize_file | Initialized MTH5 0.2.0 file /home/jovyan/earthscope-mt-course/notebooks/mth5/from_phoenix_9043.h5 in mode w
2024-10-17T11:42:17.348680-0700 | INFO | mth5.mth5 | close_mth5 | Flushing and closing /home/jovyan/earthscope-mt-course/notebooks/mth5/from_phoenix_9043.h5


## Remote Station

Here we create the remote station MTH5.  The remote station only has 2 channels, which ones are they?

In [8]:
remote_station = "remote"
remote_station_dir = source_dir.joinpath(remote_station)

phx_mth5_fn = MakeMTH5.from_phoenix(
    remote_station_dir,
    save_path=target_dir,
    mth5_filename=f"from_phoenix_{remote_station}.h5",
    sample_rates=[150, 24000],
    receiver_calibration_dict=Path(remote_station_dir.joinpath("calibrations")),
    sensor_calibration_dict=Path(remote_station_dir.joinpath("calibrations")),
)

2024-10-17T11:43:06.730981-0700 | INFO | mth5.mth5 | _initialize_file | Initialized MTH5 0.2.0 file /home/jovyan/earthscope-mt-course/notebooks/mth5/from_phoenix_remote.h5 in mode w
2024-10-17T11:43:28.751131-0700 | INFO | mth5.mth5 | close_mth5 | Flushing and closing /home/jovyan/earthscope-mt-course/notebooks/mth5/from_phoenix_remote.h5
